# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

F. Zagaria  ->  F. Zagaria  |  ['F. Zagaria']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
L. Xie  ->  Z.-L. Xie  |  ['L. Xie']
J. Li  ->  J. Li  |  ['J. Li']
Arxiv has 71 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2511.16734


extracting tarball to tmp_2511.16734...

 done.


F. Zagaria  ->  F. Zagaria  |  ['F. Zagaria']


...

extracting tarball to tmp_2511.16737...

 done.
Retrieving document from  https://arxiv.org/e-print/2511.17028


/tmp/ipykernel_2858/2822249172.py:52: LatexWarning: 2511.17028 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2511.17060
extracting tarball to tmp_2511.17060...

 done.
Retrieving document from  https://arxiv.org/e-print/2511.17313
extracting tarball to tmp_2511.17313... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.16734-b31b1b.svg)](https://arxiv.org/abs/2511.16734) | **The 12CO Gas Structures of Protoplanetary Disks in the Upper Scorpius Region**  |
|| L. Zallio, et al. -- incl., <mark>F. Zagaria</mark> |
|*Appeared on*| *2025-11-24*|
|*Comments*| *Accepted for publication in A&A. 20 pages, 7 tables, 21 figures*|
|**Abstract**|            We present measurements of key protoplanetary disk properties inferred from parametric models of ALMA 12CO spectral line visibilities. We derive gas-disk radii, integrated fluxes, optically thick emission layers, and brightness temperature profiles for the disk population of the old (4 - 14 Myr) Upper Scorpius star-forming region. We measure CO emission sizes for 37 disks with bright CO J=3-2 emission (S/N > 10 on the integrated flux; out of the 83 disks with CO detections), finding that the median radius containing 90% of the flux is ~84 au, with radii spanning from 23 up to 243 au. We report a correlation between the 12CO brightness temperatures and stellar luminosities, with a Pearson coefficient of 0.6, and we use it to prove that the 12CO optically thick emission layer primarily emanates from a region below the super-heated dust, which is optically thin to the stellar irradiation. Moreover, we derive 33 CO emission surface height profiles, finding a median aspect ratio <z/r> ~ 0.16 in a range from ~0.01 up to ~0.45 over the sample. Finally, we comment on the multiple systems in our sample, of which only some were already known. These results re-affirm how it is possible to derive bulk disk properties by modeling moderate angular resolution ALMA visibilities.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.16737-b31b1b.svg)](https://arxiv.org/abs/2511.16737) | **Zooming into the water snowline: high resolution water observations of the HL Tau disk**  |
|| M. Leemker, et al. -- incl., <mark>M. Benisty</mark> |
|*Appeared on*| *2025-11-24*|
|*Comments*| *Accepted for publication in A&A*|
|**Abstract**|            Water is one of the central molecules for the formation and habitability of planets. In particular, the region where water freezes-out, the water snowline, could be a favorable location to form planets in protoplanetary disks. We use high resolution ALMA observations to spatially resolve H$_2$O, H$^{13}$CO$^+$ and SO emission in the HL Tau disk. A rotational diagram analysis is used to characterize the water reservoir seen with ALMA and compare this to the reservoir visible at mid- and far-IR wavelengths. We find that the H$_2$O 183 GHz line has a compact central component and a diffuse component that is seen out to ~75 au. A radially resolved rotational diagram shows that the excitation temperature of the water is ~350 K independent of radius. The steep drop in the water brightness temperature outside the central beam of the observations where the emission is optically thick is consistent with the water snowline being located inside the central beam ($\lesssim 6$ au) at the height probed by the observations. Comparing the ALMA lines to those seen at shorter wavelengths shows that only 0.02%-2% of the water reservoir is visible at mid- and far-IR wavelengths, respectively, due to optically thick dust hiding the emission whereas 35-70% is visible with ALMA. An anti-correlation between the H$_2$O and H$^{13}$CO$^+$ emission is found but this is likely caused by optically thick dust hiding the H$^{13}$CO$^+$ emission in the disk center. Finally, we see SO emission tracing the disk and for the first time in SO a molecular outflow and the infalling streamer out to ~2". The velocity structure hints at a possible connection between the SO and the H$_2$O emission. Spatially resolved observations of H$_2$O lines at (sub-)mm wavelengths provide valuable constraints on the location of the water snowline, while probing the bulk of the gas-phase reservoirs.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.17060-b31b1b.svg)](https://arxiv.org/abs/2511.17060) | **Comparing galaxy merger orbits in hydrodynamical simulation and in dark-matter-only simulation**  |
|| Y. Pu, L. Wang, G. Zeng, <mark>L. Xie</mark> |
|*Appeared on*| *2025-11-24*|
|*Comments*| *18 pages, 8 figures, submitted to RAA*|
|**Abstract**|            To investigate how the presence of baryons in simulations affects galaxy merger orbits, we compare in detail the merger timescales and orbits of the matched merger pairs in TNG100 hydrodynamical simulations and their corresponding dark-matter-only simulations, for different resolution levels. Compared with the mergers in the TNG100-1-Dark simulation without baryons, the matched mergers in the TNG100-1 simulation have similar infall time, but have statistically earlier merger times and therefore shorter merger timescales. The merger orbits for the matched pairs in the TNG100-1 and the TNG100-1-Dark simulations are similar right after infall, and both evolve to more head-on orbits at final stages, with smaller changes in the hydrodynamical simulation. In the final 2 Gyr before merger, the collision angles that represent merger orbits quantitatively are smaller in TNG100-1 than those in TNG100-1-Dark, by around 6$^\circ$ to 10$^\circ$, depending on the mass ratios and galaxy masses investigated. Our results demonstrate that the presence of baryons accelerates a bit the merger processes, and results in more spiral-in orbits for both major and minor mergers in galaxies with various stellar masses. These effects are less obvious in simulations with lower resolutions.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.17313-b31b1b.svg)](https://arxiv.org/abs/2511.17313) | **On the baryon budget in the X-ray-emitting circumgalactic medium of Milky Way-mass galaxies**  |
|| Y. Zhang, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-11-24*|
|*Comments*| *12 pages, 3 figures, 3 tables. Accepted by A&A*|
|**Abstract**|            Recent observations with SRG/eROSITA have revealed the average X-ray surface brightness profile of the X-ray-emitting circumgalactic medium (CGM) around Milky Way (MW)-mass galaxies, offering valuable insights into the baryon mass in these systems. However, the estimation of the baryon mass depends critically on several assumptions regarding the gas density profile, temperature, metallicity, and the underlying halo mass distribution. Here, we assess how these assumptions affect the inferred baryon mass of the X-ray-emitting CGM in MW-mass galaxies, based on the stacked eROSITA signal. We find that variations in temperature profiles and uncertainties in the halo mass introduce the dominant sources of uncertainty, resulting in X-ray-emitting baryon mass estimates that vary by nearly a factor of four ($0.8-3.5\times10^{11} M_\odot$). Assumptions about metallicity contribute an additional uncertainty of approximately $50\%$. We emphasize that accurate X-ray spectral constraints on gas temperature and metallicity, along with careful modeling of halo mass uncertainty, are essential for accurately estimating the baryon mass for MW-mass galaxies. Future X-ray microcalorimeter missions will be crucial for determining the hot CGM properties and closing the baryon census at the MW-mass scale.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.17028-b31b1b.svg)](https://arxiv.org/abs/2511.17028) | **Multiphase Astrophysics to Unveil the Virgo Environment (MAUVE)**  |
|| B. Catinella, et al. -- incl., <mark>E. Schinnerer</mark> |
|*Appeared on*| *2025-11-24*|
|*Comments*| *4 pages, 3 figures. Published in the ESO Messenger 195*|
|**Abstract**|            The Multiphase Astrophysics to Unveil the Virgo Environment (MAUVE) project is a multi-facility programme exploring how dense environments transform galaxies. Combining a VLT/MUSE P110 Large Programme and ALMA observations of 40 late-type Virgo Cluster galaxies, MAUVE resolves star formation, kinematics, and chemical enrichment within their molecular gas discs. A key goal is to track the evolution of cold gas that survives in the inner regions of satellites after entering the cluster, and how it evolves across different infall stages. With its high spatial resolution -- probing down to the physical scales of giant molecular cloud complexes -- and multiphase synergy, MAUVE aims to offer a time-resolved view of environmental quenching and set a new benchmark for cluster galaxy studies.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2511.16734/./pictures/core_plots/T10_vs_logL_fit_spheric_w_Teq_Ts_colored.png', 'tmp_2511.16734/./pictures/core_plots/2MASS_J16213469-2612269.png', 'tmp_2511.16734/./pictures/core_plots/z_over_r_KM.png']
copying  tmp_2511.16734/./pictures/core_plots/T10_vs_logL_fit_spheric_w_Teq_Ts_colored.png to _build/html/
copying  tmp_2511.16734/./pictures/core_plots/2MASS_J16213469-2612269.png to _build/html/
copying  tmp_2511.16734/./pictures/core_plots/z_over_r_KM.png to _build/html/
exported in  _build/html/2511.16734.md
    + _build/html/tmp_2511.16734/./pictures/core_plots/T10_vs_logL_fit_spheric_w_Teq_Ts_colored.png
    + _build/html/tmp_2511.16734/./pictures/core_plots/2MASS_J16213469-2612269.png
    + _build/html/tmp_2511.16734/./pictures/core_plots/z_over_r_KM.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\gr}[1]{\textbf{GR: \color{cyan} #1}}$
$\newcommand{\thetable}{A.\arabic{table}}$
$\newcommand{\thefigure}{B.\arabic{figure}}$</div>



<div id="title">

# The $^{12}$CO Gas Structures of Protoplanetary Disks in the Upper Scorpius Region  

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.16734-b31b1b.svg)](https://arxiv.org/abs/2511.16734)<mark>Appeared on: 2025-11-24</mark> -  _Accepted for publication in A&A. 20 pages, 7 tables, 21 figures_

</div>
<div id="authors">

L. Zallio, et al. -- incl., <mark>F. Zagaria</mark>

</div>
<div id="abstract">

**Abstract:** We present measurements of key protoplanetary disk properties inferred from parametric models of ALMA $^{12}$ CO spectral line visibilities. We derive gas-disk radii, integrated fluxes, optically thick emission layers, and brightness temperature profiles for the disk population of the old ( $4 - 14$ Myr) Upper Scorpius star-forming region. We measure CO emission sizes for 37 disks with bright CO $J=3-2$ emission (S/N > 10 on the integrated flux; out of the 83 disks with CO detections), finding that the median radius containing 90 \% of the flux is $\sim84$ au, with radii spanning from $23$ up to $243$ au.We report a correlation between the $^{12}$ CO brightness temperatures and stellar luminosities, with a Pearson coefficient of 0.6, and we use it to prove that the $^{12}$ CO optically thick emission layer primarily emanates from a region below the super-heated dust, which is optically thin to the stellar irradiation. Moreover, we derive 33 CO emission surface height profiles, finding a median aspect ratio $\langle z/r \rangle \sim 0.16$ in a range from $\sim0.01$ up to $\sim0.45$ over the sample. Finally, we comment on the multiple systems in our sample, of which only some were already known.These results re-affirm how it is possible to derive bulk disk properties by modeling moderate angular resolution ALMA visibilities.

</div>

<div id="div_fig1">

<img src="tmp_2511.16734/./pictures/core_plots/T10_vs_logL_fit_spheric_w_Teq_Ts_colored.png" alt="Fig5" width="100%"/>

**Figure 5. -** The correlation between the $^{12}$CO brightness temperature at a distance of 10 au from the central star and stellar luminosity. The values of $T_{b,10}$ and $\log_{10}\bigl(\text{L}_*\bigr)$ were taken from Tables \ref{Table:Fits_A}, \ref{Table:Fits_B} and Empey et al., in prep., respectively. The values of $\text{T}_{b,10}$ were converted from cylindrical coordinates to spherical coordinates using the vertical height profile values reported in Table \ref{Table:Fits_A}. The values of F$_{\text{cont}}$ were taken from \cite{Carpenter2025}. (*fig:Tb_L*_corr*)

</div>
<div id="div_fig2">

<img src="tmp_2511.16734/./pictures/core_plots/2MASS_J16213469-2612269.png" alt="Fig18" width="100%"/>

**Figure 18. -** Residual plot for J16213469-2612269. In the first row, we show the channel maps of the data; in the second row, we show the channel maps of the best-fit model; in the third row we show the channel maps of the residuals (data - model). The residual plots for all the disks analyzed in this work can be found in Appendix \ref{appendix:residuals}, and on \texttt{GitHub}\protect\hyperref[footnote:github]{\textsuperscript{\ref*{footnote:github}}}. (*fig:residuals_example*)

</div>
<div id="div_fig3">

<img src="tmp_2511.16734/./pictures/core_plots/z_over_r_KM.png" alt="Fig4" width="100%"/>

**Figure 4. -** Kaplan-Meier estimator of the radially averaged $^{12}$CO $ \langle z/r \rangle $ values for our sample (in blue), compared with those of \cite{Galloway2025}(in orange), and the combined works of \cite{Law_2022, Law_2023, Law_2024}(in red). (*fig:z_over_r_histo*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.16734"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

132  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

16  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
